In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors


make sure to rename the columns by removing é


# Exploratory Data Analysis (EDA) for Packages and Receptacle Datasets

In [ ]:
# Load datasets
packages_df = pd.read_csv('../data/raw/packages_data_2023_2025.csv',delimiter=';')
receptacle_df = pd.read_csv('../data/raw/receptacle_data_2023_2025.csv',delimiter=';')

In [ ]:
# Rename for consistency
receptacle_df.rename(columns={
    'EVENT_TYPECD': 'EVENT_TYPE_CD',
    'nextetablissement_postal': 'next_etablissement_postal'
}, inplace=True)

In [ ]:
packages_df.shape, receptacle_df.shape

In [ ]:
#convert dates to dateTime for compatibility
packages_df['date'] = pd.to_datetime(packages_df['date'])
receptacle_df['date'] = pd.to_datetime(receptacle_df['date'])

In [ ]:
packages_df.info()


In [ ]:
receptacle_df.info()


In [ ]:
packages_df.head(n=5)

In [ ]:
receptacle_df.head(n=5)

 Initial Observations
- both datasets cover the period from 2023 to 2025
- we have no target variable in either dataset
- for packages dataset:
    - 6 features in total with 5 categorical and 1 numerical
    - MAILITM_FID is unique identifier for each package
    - RECPTCL_FID is foreign key linking to receptacle dataset
    - etablissement_postal and next_etablissement_postal have some null values
- for receptacle dataset:
    - 5 features in total with 4 categorical and 1 numerical
    - RECPTCL_FID is unique identifier for each receptacle
    - EVENT_TYPE_CD has some null values

In [ ]:
for column in packages_df.columns:
    print(f'{column} has {packages_df[column].nunique()} unique values.')
    print(f'{column} has {packages_df[column].isnull().sum()} null values.')
    print("\n")


In [ ]:
for column in receptacle_df.columns:
    print(f'{column} has {receptacle_df[column].nunique()} unique values.')
    print(f'{column} has {receptacle_df[column].isnull().sum()} null values.')
    print("\n")

we notice the following:<br>
- receptacle dataset has more unique values for RECPTCL_FID than packages dataset, indicating one-to-many relationship<br>
- MAILITM_FID is unique in packages dataset.<br>
- packages dataset have more unique date values than receptacle dataset.<br>
- both datasets have null values in etablissement_postal and next_etablissement_postal columns. This requires processing later on<br>
- packages dataset has more unique values in the next_etablissement_postal column compared to receptacle dataset but also more null values. **further investigation is needed to understand why**<br>


In [ ]:
packages_df.describe()

In [ ]:
receptacle_df.describe()

for EVENT_TYPE_CD we notice different range of values for packages and receptacle datasets indicating different types of events.<br>

#### Visualizations
for now we will visualize the distribution of EVENT_TYPE_CD in both datasets.<br>



In [ ]:
# Distribution of EVENT_TYPE_CD in packages dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=packages_df,x='EVENT_TYPE_CD')
plt.title('distribution of EVENT_TYPE_CD in packages dataset')
plt.xlabel('EVENT_TYPE_CD')
plt.ylabel('count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Distribution of EVENT_TYPE_CD in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacle_df,x='EVENT_TYPE_CD')
plt.title('distribution of EVENT_TYPE_CD in Receptacle dataset')
plt.xlabel('EVENT_TYPE_CD')
plt.ylabel('count')
plt.xticks(rotation=45)
plt.show()

for etablissement_postal and next_etablissement_postal I will start with visualizing the receptacle dataset since the packages dataset has a lot of unique values<br>

In [ ]:
# Distribution of etablissement_postal in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacle_df,x='etablissement_postal')
plt.title('distribution of etablissement_postal in Receptacle dataset')
plt.xlabel('etablissement_postal')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Distribution of next_etablissement_postal in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacle_df,x='next_etablissement_postal')
plt.title('distribution of next_etablissement_postal in Receptacle dataset')
plt.xlabel('next_etablissement_postal')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# 1.  Top 10 locations to keep the map small
top_locs = receptacle_df['etablissement_postal'].value_counts().head(10).index
top_next = receptacle_df['next_etablissement_postal'].value_counts().head(10).index

# 2. include only these top locations
filtered_df = receptacle_df[
    (receptacle_df['etablissement_postal'].isin(top_locs)) &
    (receptacle_df['next_etablissement_postal'].isin(top_next))
]

# 3. Create a Matrix (Cross-tabulation)
matrix = pd.crosstab(filtered_df['etablissement_postal'], filtered_df['next_etablissement_postal'])

# 4. Plot Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matrix, cmap='Reds', annot=True, fmt='d', linewidths=.5)
plt.title("Most Frequent Routes (Origin vs Destination) - Receptacle Dataset")
plt.xlabel("Next Destination")
plt.ylabel("Current Location")
plt.show()

we notice that some etablissements have significantly higher traffic compared to others, indicating  major distribution centers.<br>

for etablissement_postal and next_etablissement_postal we will create a heatmap to visualize the flow between current location and next destination.<br>
Count of parcels moving from A to B to see the density of connections between them

In [ ]:
# 1.  Top 10 locations to keep the map small
top_locs = packages_df['etablissement_postal'].value_counts().head(10).index
top_next = packages_df['next_etablissement_postal'].value_counts().head(10).index

# 2. include only these top locations
filtered_df = packages_df[
    (packages_df['etablissement_postal'].isin(top_locs)) &
    (packages_df['next_etablissement_postal'].isin(top_next))
]

# 3. Create a Matrix (Cross-tabulation)
matrix = pd.crosstab(filtered_df['etablissement_postal'], filtered_df['next_etablissement_postal'])

# 4. Plot Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matrix, cmap='Reds', annot=True, fmt='d', linewidths=.5)
plt.title("Most Frequent Routes (Origin vs Destination) - Packages Dataset")
plt.xlabel("Next Destination")
plt.ylabel("Current Location")
plt.show()

In [ ]:
# Count packages per location
location_counts = packages_df['etablissement_postal'].value_counts().reset_index()
location_counts.columns = ['Location', 'Volume']

# keep only top 20 busiest centers
plt.figure(figsize=(12, 6))
sns.barplot(x='Volume', y='Location', data=location_counts.head(20), palette='viridis')
plt.title("Top 20 Busiest Postal Centers")
plt.xlabel("Number of Packages")
plt.ylabel("Center ID")
plt.show()

we notice the same pattern as before with some etablissements having significantly higher traffic compared to others.<br>

In [ ]:

# 1. Extract Time Features
packages_df['hour'] = packages_df['date'].dt.hour
packages_df['day_of_week'] = packages_df['date'].dt.day_name()

# 2. Create a Pivot Table (Cross-tabulation)
# Rows = Day, Cols = Hour, Values = Count of Scans
heatmap_data = pd.crosstab(
    packages_df['day_of_week'],
    packages_df['hour']
)

days_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
heatmap_data = heatmap_data.reindex(days_order)

# 3. Plot
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlOrRd', linewidths=.5, annot=False)
plt.title("Package Scan Activity by Day and Hour")
plt.xlabel("Hour of Day (0-23)")
plt.ylabel("Day of Week")
plt.show()

we notice that the busiest times for package scans are during weekdays, particularly from mid-morning to late afternoon.<br>

In [ ]:
# Merge datasets on RECPTCL_FID to analyze relationship
merged_df = pd.merge(
    packages_df,
    receptacle_df,
    on='RECPTCL_FID',
    how='left',
    suffixes=('_package', '_receptacle')
)

In [ ]:
merged_df.head(n=20)

In [ ]:
#number of packages per receptacle
packages_per_receptacle = packages_df.groupby('RECPTCL_FID')['MAILITM_FID'].nunique()
packages_per_receptacle.head()


# Feature Extraction 

In [ ]:

#Use raw
# packages_df = pd.read_csv('../data/raw/packages_data_2023_2025.csv',delimiter=';')
# receptacle_df = pd.read_csv('../data/raw/receptacle_data_2023_2025.csv',delimiter=';')

#this is to use the preprocessed one
packages_df = pd.read_csv('../data/interim/clean_packages_df.csv',delimiter=',')
receptacle_df = pd.read_csv('../data/interim/clean_receptacles_df.csv',delimiter=',')

### stripe white spaces from IDs

In [ ]:

packages_df['RECPTCL_FID'] = packages_df['RECPTCL_FID'].str.strip()
packages_df['MAILITM_FID'] = packages_df['MAILITM_FID'].str.strip()
receptacle_df['RECPTCL_FID'] = receptacle_df['RECPTCL_FID'].str.strip()

### Check RECPTCL_FID and MAILITM_FID having same length formats
if yes then we can split them into meaningfull parts

In [ ]:
same=1
print("\n=== RECPTCL_FID  ===")
print(f"testing if the lengths of RECPTCL_FID values are all the same:")
for val in packages_df['RECPTCL_FID'].values:
    if len(str(val)) != 29 :
        print(f"  {val} (length: {len(str(val))})")
        same=0
        break
if same==1:
    print('all same length' )

print("\n=== MAILITM_FID  ===")
print(f"testing if the lengths of MAILITM_FID values are all the same:")
for val in packages_df['MAILITM_FID'].values:
    if len(str(val)) != 13 :
        print(f"  {val} (length: {len(str(val))})")
        same=0
        break
if same==1:
    print ('all same length' )



### RECPTCL_FID Analysis
- **Format:** 29-character string (e.g., `USORDADZALGDAUN30050001900005`)
- **Data Quality:** No null values (1,000,000) | 215,867 unique values in receptacle dataset and 45306 unique values in packages dataset
- **Extractable Features:**
  - Origin Country (2 chars): US, FR, AE, etc.
  - Destination Country (2 chars): DZ, AI, AA, etc.

### MAILITM_FID Analysis
- **Format:** 13-character string according to the S10-12 patern (e.g., `CA000132868US`, `CA000340856PK`)
- **Data Quality:** No null values (1,000,000 packages)
- **Extractable Features:**
  - Service Indicator (2 chars): CA, etc.
  - Serial Number (8 chars): 00013286, 00034085, etc.
  - Check Digit (1 char): 8, 6, etc.
  - Country Code (3 chars, right-stripped): US, PK.


## Definition of the parser funtions
These functions are responsible for spliting the IDs into parts

In [ ]:
def parse_recptcl_fid(id_str):
    origin_country = id_str[0:2]
    destination_country = id_str[6:8]
    return origin_country, destination_country

def parse_mailitm_fid(id_str):
    service_indicator = id_str[0:2]
    serial_number = id_str[2:11]
    country_code = id_str[11:14].strip()
    return service_indicator, serial_number, country_code


### Apply parsing functions

In [ ]:
# parsed_receptacle_df = receptacle_df.copy()
# parsed_receptacle_df[['origin_country', 'destination_country']] = parsed_receptacle_df['RECPTCL_FID'].apply(lambda x: pd.Series(parse_recptcl_fid(x)))

# Faster method:
parsed_data = list(receptacle_df['RECPTCL_FID'].apply(parse_recptcl_fid))

# Assign to new columns by creating a temporary DataFrame
parsed_receptacle_df = receptacle_df.copy()
parsed_receptacle_df[['origin_country', 'destination_country']] = pd.DataFrame(
    parsed_data, index=receptacle_df.index
)

In [ ]:
# parsed_packages_df = packages_df.copy()
# parsed_packages_df[['service_indicator', 'serial_number', 'country_code']] = parsed_packages_df['MAILITM_FID'].apply(lambda x: pd.Series(parse_mailitm_fid(x)))
# parsed_packages_df[['origin_country','destination_country']] = parsed_packages_df['RECPTCL_FID'].apply(lambda x: pd.Series(parse_recptcl_fid(x)))

# Faster method:
parsed_packages_df = packages_df.copy()

# 1. Optimize MAILITM_FID parsing
mailitm_data = list(parsed_packages_df['MAILITM_FID'].apply(parse_mailitm_fid))
parsed_packages_df[['service_indicator', 'serial_number', 'country_code']] = pd.DataFrame(
    mailitm_data, index=parsed_packages_df.index
)

# 2. Optimize RECPTCL_FID parsing
recptcl_data = list(parsed_packages_df['RECPTCL_FID'].apply(parse_recptcl_fid))
parsed_packages_df[['origin_country', 'destination_country']] = pd.DataFrame(
    recptcl_data, index=parsed_packages_df.index
)

### show samples of new parsed data

In [ ]:

print("=== packages_df sample with new parsed columns ===")
parsed_packages_df.head()



In [ ]:
print("\n=== receptacle_df sample with new parsed columns ===")
parsed_receptacle_df.head()

# Analysis of the extrcted features


## 1. parsed_packages_df

In [ ]:

print("\n--- Unique Value Counts for parsed_packages_df ---")
print("\nFor receptacle FID parsing:")
print(f"Unique origin_country values: {parsed_packages_df['origin_country'].nunique()}")
print(f"Unique destination_country values: {parsed_packages_df['destination_country'].nunique()}")

print("="*50)
print("\nFor mail item FID parsing:")
print(f"Unique service_indicator values: {parsed_packages_df['service_indicator'].nunique()}")
print(f"Unique country_code values: {parsed_packages_df['country_code'].nunique()}")



## 2. parsed_receptacle_df 

In [ ]:
print("\n--- Unique Value Counts for parsed_receptacle_df ---")
print(f"Unique origin_country values: {parsed_receptacle_df['origin_country'].nunique()}")
print(f"Unique destination_country values: {parsed_receptacle_df['destination_country'].nunique()}")


## List values of the new columns obtained from receptacle FID parsing for both parsed dataframes

### 1. for parsed_packages_df

In [ ]:
#listing the values 
print("\n--- Values of origin_country ---")
print(parsed_packages_df['origin_country'].unique())

print("\n--- Values of destination_country ---")
print(parsed_packages_df['destination_country'].unique())


### 2. for parsed_receptacle_df

In [ ]:
#listing the values 
print("\n--- Values of origin_country ---")
print(parsed_receptacle_df['origin_country'].unique())

print("\n--- Values of destination_country ---")
print(parsed_receptacle_df['destination_country'].unique())


### Do the intersection of origin_country of both dataframes

In [ ]:
#do the intersection of origin_country values in both parsed datasets
packages_origin_countries = set(parsed_packages_df['origin_country'].unique())
receptacle_origin_countries = set(parsed_receptacle_df['origin_country'].unique())
common_origin_countries = packages_origin_countries.intersection(receptacle_origin_countries)
print("number of common origin_country values in both parsed datasets:", len(common_origin_countries))
print(f"\nCommon origin_country values in both paesed datasets: ")
print(common_origin_countries)
# remaining ones 
remaining_in_packages = packages_origin_countries - common_origin_countries
remaining_in_receptacle = receptacle_origin_countries - common_origin_countries
print(f"Remaining origin_country values only in parsed_packages_df:")
print(remaining_in_packages)
print(f"Remaining origin_country values only in parsed_receptacle_df:")
print(remaining_in_receptacle )

### list the values of both service indicators and country code

### 1. service indicators

In [ ]:
print("\n--- Values of service_indicator ---")
print(parsed_packages_df['service_indicator'].unique())



 we can see that there are values that don't follow the standards in the S10-12 format so we need to handle that correctly

In [ ]:
#transform country_code to uppercase for consistency
parsed_packages_df['service_indicator'] = parsed_packages_df['service_indicator'].str.upper()
print("values of service_indicator after transformation to uppercase:")
print(parsed_packages_df['service_indicator'].unique())
print("number of unique service indicators after transformation:", parsed_packages_df['service_indicator'].nunique())


### 2. country code

In [ ]:
print("\n--- Values of country codes ---")

print(parsed_packages_df['country_code'].unique())

we can see that many values for the country codes are numbers instead of ISO 3166-1 format these values should be replaced by the values of origin country gotten from the receptacle when doing the preprocessing

### replace them with the correct origin country code

In [ ]:
# 1. Vectorized string capitalization
parsed_packages_df['country_code'] = parsed_packages_df['country_code'].str.upper()

# 2. Vectorized comparison to find mismatches
mismatch_mask = parsed_packages_df['origin_country'] != parsed_packages_df['country_code']

# 3. Count the Trues
count = mismatch_mask.sum()

print(f"Number of rows where origin_country does not match country_code: {count}")

### replace them

In [ ]:
# Use .loc to find rows where they don't match, and update only the 'country_code' column
parsed_packages_df.loc[parsed_packages_df['origin_country'] != parsed_packages_df['country_code'], 'country_code'] = parsed_packages_df['origin_country']

In [ ]:
#print the unique values again
print("\n--- Values of country codes after correction ---")
print(parsed_packages_df['country_code'].unique())
print("number of unique country codes after correction:", parsed_packages_df['country_code'].nunique())

## visualization of Origin Country distribution according to number of packages

### 1. for the parsed_packages_df

In [ ]:
origin_country_counts = parsed_packages_df['origin_country'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(origin_country_counts.index, origin_country_counts.values, color='steelblue')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Origin Country', fontsize=11)
plt.title('Top 20 Origin Countries by packages count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 2. for the parsed_receptacle_df

In [ ]:
origin_country_counts = parsed_receptacle_df['origin_country'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(origin_country_counts.index, origin_country_counts.values, color='steelblue')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Origin Country', fontsize=11)
plt.title('Top 20 Origin Countries by receptacle count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Visualiation of the service indicator

In [ ]:
service_indicator_count = parsed_packages_df['service_indicator'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(service_indicator_count.index, service_indicator_count.values, color='mediumseagreen')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Service Indicator', fontsize=11)
plt.title('Top 20 Service Indicators by packages count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Origin–Destination Flow Analysis

This section investigates the flow of receptacles and packages from origin countries to destination. We examine:
- packages count by origin country
- Top origin countries delivering to each destination
- Visual representation 

### 1. parsed_packages_df

In [ ]:
# packages count by origin country
origin_country_volume = parsed_packages_df['origin_country'].value_counts()
print("\n--- Packages count by Origin Country ---")
print(origin_country_volume.head(15))

# Create origin_country × destination matrix
flow_matrix = pd.crosstab(parsed_packages_df['origin_country'], 
                           parsed_packages_df['destination_country'])

# Keep only top 10 origin countries and top 10 destination countries for readability
top_origins = parsed_packages_df['origin_country'].value_counts().head(10).index
top_arrivals = parsed_packages_df['destination_country'].value_counts().head(10).index

flow_matrix_top = flow_matrix.loc[top_origins, top_arrivals]

print("\n--- Origin Country × destination Country (Top 10 × Top 10) ---")
print(flow_matrix_top)


In [ ]:
# Visualize origin × destination country flow
plt.figure(figsize=(14, 8))
sns.heatmap(flow_matrix_top, cmap='YlOrRd', annot=True, fmt='d', 
            cbar_kws={'label': 'packages Count'}, linewidths=0.5)
plt.title('packages Flow: Origin Country × destination country (Top 10 × Top 10)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Arrival Hub', fontsize=12)
plt.ylabel('Origin Country', fontsize=12)
plt.tight_layout()
plt.show()


### 2. parsed_receptacle_df

In [ ]:
# receptacle count by origin country
origin_country_receptacle_volume = parsed_receptacle_df['origin_country'].value_counts()
print("\n--- Receptacle count by Origin Country ---")
print(origin_country_receptacle_volume.head(15))

destination__receptacle_volume=parsed_receptacle_df['destination_country'].value_counts()
print("\n--- Receptacle count by destination ---")
print(destination__receptacle_volume.head(15))

# Create origin_country × destination matrix
receptacle_flow_matrix = pd.crosstab(parsed_receptacle_df['origin_country'], 
                           parsed_receptacle_df['destination_country'])

# Keep only top 10 origin countries and top 10 destination countries for readability
receptacle_top_origins = parsed_receptacle_df['origin_country'].value_counts().head(10).index
receptacle_top_arrivals = parsed_receptacle_df['destination_country'].value_counts().head(10).index

receptacle_flow_matrix_top = receptacle_flow_matrix.loc[receptacle_top_origins, receptacle_top_arrivals]

print("\n--- Origin Country × destination Country (Top 10 × Top 10) ---")
print(receptacle_flow_matrix_top)


In [ ]:
# Visualize origin × destination country flow
plt.figure(figsize=(14, 8))
sns.heatmap(receptacle_flow_matrix_top, cmap='YlOrRd', annot=True, fmt='d', 
            cbar_kws={'label': 'receptacles Count'}, linewidths=0.5)
plt.title('receptacles Flow: Origin Country × destination country (Top 10 × Top 10)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Arrival Hub', fontsize=12)
plt.ylabel('Origin Country', fontsize=12)
plt.tight_layout()
plt.show()


### create pairs (origin, destination) for more detailed analysis

In [ ]:
parsed_packages_df['origin_destination'] = parsed_packages_df['origin_country'] + '_' + parsed_packages_df['destination_country']
parsed_receptacle_df['origin_destination'] = parsed_receptacle_df['origin_country'] + '_' + parsed_receptacle_df['destination_country']


listing the obtained values 

In [ ]:
print('(origin_destination) pairs obtained for ')
print("\nfor parsed_packages_df :")
print("\nNumber of unique values", parsed_packages_df['origin_destination'].nunique())
print("\nValues : ")
print(parsed_packages_df['origin_destination'].unique())
print("\nfor parsed_receptacles_df :")
print("\nNumber of unique values", parsed_receptacle_df['origin_destination'].nunique())
print("\nValues : ")
print(parsed_receptacle_df['origin_destination'].unique())


### visualization of obtained results 

### 1. parsed_packages_df

In [ ]:
origin_dest_counts = parsed_packages_df['origin_destination'].value_counts()
top_origin_dest_counts= origin_dest_counts.head(15)
plt.figure(figsize=(14, 7))
top_origin_dest_counts.plot(kind='bar')
plt.title('Histogram of (origin, destination) Pairs Based on Package Counts')
plt.xlabel('Origin_Destination')
plt.ylabel('Number of Packages')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



### 2. parsed_receptacle_df

In [ ]:
origin_dest_counts = parsed_receptacle_df['origin_destination'].value_counts()
top_origin_dest_counts = origin_dest_counts.head(15)

plt.figure(figsize=(14, 7))
top_origin_dest_counts.plot(kind='bar')
plt.title('Histogram of (origin, destination) Pairs Based on receptacle Counts')
plt.xlabel('Origin_Destination')
plt.ylabel('Number of receptacles')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


## origin_destination X etablissments analysis 

## 1. current etablissment 

### a. parsed_packages_df

In [ ]:
# For parsed_packages_df: visualize the histogram of counts by (origin_destination, etablissement_postal) pairs

# First, select top N most common (origin_destination, etablissement_postal) pairs for clarity
pair_counts_current = parsed_packages_df.groupby(['origin_destination', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_current = pair_counts_current.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_current,
    x='count',
    y=top_pairs_current.apply(lambda x: f"{x['origin_destination']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Current Etablissement) Pairs by Package Counts')
plt.xlabel('Number of Packages')
plt.ylabel('(Origin_Destination | Current Etablissement)')
plt.tight_layout()
plt.show()


### b. parsed_receptacle_df

In [ ]:
# For parsed_receptacle_df: visualize the histogram of counts by (origin_destination, etablissement_postal) pairs

# First, select top N most common (origin_destination, etablissement_postal) pairs for clarity
pair_counts_current_receptacle = parsed_receptacle_df.groupby(['origin_destination', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_current_receptacle = pair_counts_current_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_current_receptacle,
    x='count',
    y=top_pairs_current_receptacle.apply(lambda x: f"{x['origin_destination']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Destination | Current Etablissement)')
plt.tight_layout()
plt.show()


we can see that the ETAB0002 is dominating and we remark that when the destination is DZ
we'll try to confirm that by taking into consideration the destination only


In [ ]:
# For parsed_receptacle_df: visualize the histogram of counts by (destination_country, etablissement_postal) pairs

# First, select top N most common (destination_country, etablissement_postal) pairs for clarity
pair_counts_dest_receptacle = parsed_receptacle_df.groupby(['destination_country', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_dest_receptacle = pair_counts_dest_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_dest_receptacle,
    x='count',
    y=top_pairs_dest_receptacle.apply(lambda x: f"{x['destination_country']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Destination Country, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Destination Country | Current Etablissement)')
plt.tight_layout()
plt.show()


### This is to test the origin 

In [ ]:
# For parsed_receptacle_df: visualize the histogram of counts by (origin_country, etablissement_postal) pairs

# First, select top N most common (origin_country, etablissement_postal) pairs for clarity
pair_counts_origin_receptacle = parsed_receptacle_df.groupby(['origin_country', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_origin_receptacle = pair_counts_origin_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_origin_receptacle,
    x='count',
    y=top_pairs_origin_receptacle.apply(lambda x: f"{x['origin_country']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin Country, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin Country | Current Etablissement)')
plt.tight_layout()
plt.show()


most of the values with ETAB0002 values are european countries in addition to AE and China(CN)

## 2. Next etablissement

### a. parsed_packages_df

In [ ]:
# For parsed_packages_df: visualize the histogram of counts by (origin_destination, next_etablissement_postal)

# First, select top N most common (origin_destination, next_etablissement_postal) pairs for clarity
pair_counts = parsed_packages_df.groupby(['origin_destination', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs = pair_counts.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs,
    x='count',
    y=top_pairs.apply(lambda x: f"{x['origin_destination']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Next Etablissement) Pairs by Package Counts')
plt.xlabel('Number of Packages')
plt.ylabel('(Origin_Destination | Next Etablissement)')
plt.tight_layout()
plt.show()


### b. parsed_receptacle_df

In [ ]:
# For parsed_receptacle_df: visualize the histogram of counts by (origin_destination, next_etablissement_postal)

# First, select top N most common (origin_destination, next_etablissement_postal) pairs for clarity
pair_counts_recept = parsed_receptacle_df.groupby(['origin_destination', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_recept = pair_counts_recept.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_recept,
    x='count',
    y=top_pairs_recept.apply(lambda x: f"{x['origin_destination']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Destination | Next Etablissement)')
plt.tight_layout()
plt.show()



Do for origin and for destination separately

In [ ]:

# For parsed_receptacle_df: visualize the histogram of counts by destination_country and next_etablissement_postal

# First, select top N most common (destination_country, next_etablissement_postal) pairs for clarity
pair_counts_dest = parsed_receptacle_df.groupby(['destination_country', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_dest = pair_counts_dest.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_dest,
    x='count',
    y=top_pairs_dest.apply(lambda x: f"{x['destination_country']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Destination Country, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Destination_Country | Next Etablissement)')
plt.tight_layout()
plt.show()


In [ ]:
# For parsed_receptacle_df: visualize the histogram of counts by origin_country and next_etablissement_postal

# First, select top N most common (origin_country, next_etablissement_postal) pairs for clarity
pair_counts_origin = parsed_receptacle_df.groupby(['origin_country', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_origin = pair_counts_origin.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_origin,
    x='count',
    y=top_pairs_origin.apply(lambda x: f"{x['origin_country']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin Country, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Country | Next Etablissement)')
plt.tight_layout()
plt.show()


from the analysis we can see that there are some etablissments that get congested forming a sort of loop (ETAB0030, ETAB0002, ETAB0006)

### Time analysis regarding origin

### 1. parsed_packages_df

In [ ]:
# Group the parsed_packages_df by date and origin_country to count packages per month per origin country
parsed_packages_df['date'] = pd.to_datetime(parsed_packages_df['date'])

# Group by origin_country and resample by month
ts_by_origin = parsed_packages_df.groupby(['origin_country', pd.Grouper(key='date', freq='ME')])['MAILITM_FID'].count().reset_index()

# Choose top 5 origin countries by total count for plotting
top_origin = parsed_packages_df['origin_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for origin in top_origin:
    ts = ts_by_origin[ts_by_origin['origin_country'] == origin]
    plt.plot(ts['date'], ts['MAILITM_FID'], marker='o', label=origin)

plt.title('Monthly Package count by Origin Country')
plt.xlabel('Date')
plt.ylabel('Number of Packages')
plt.legend(title='Origin Country')
plt.tight_layout()
plt.show()


### 2. parsed_receptacle_df

In [ ]:
# Group the parsed_receptacle_df by date and origin_country to count receptacles per month per origin country
parsed_receptacle_df['date'] = pd.to_datetime(parsed_receptacle_df['date'])

# Group by origin_country and resample by month
ts_by_origin_recept = parsed_receptacle_df.groupby(['origin_country', pd.Grouper(key='date', freq='ME')])['RECPTCL_FID'].count().reset_index()

# Choose top 5 origin countries by total count for plotting
top_origin_recept = parsed_receptacle_df['origin_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for origin in top_origin_recept:
    ts = ts_by_origin_recept[ts_by_origin_recept['origin_country'] == origin]
    plt.plot(ts['date'], ts['RECPTCL_FID'], marker='o', label=origin)

plt.title('Monthly Receptacle count by Origin Country')
plt.xlabel('Date')
plt.ylabel('Number of Receptacles')
plt.legend(title='Origin Country')
plt.tight_layout()
plt.show()


### Time analysis by destination

In [ ]:
# Group the parsed_packages_df by date and destination_country to count packages per month per destination country
parsed_packages_df['date'] = pd.to_datetime(parsed_packages_df['date'])

# Group by destination_country and resample by month
ts_by_dest = parsed_packages_df.groupby(['destination_country', pd.Grouper(key='date', freq='ME')])['MAILITM_FID'].count().reset_index()

# Choose top 5 destination countries by total count for plotting
top_dest = parsed_packages_df['destination_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for dest in top_dest:
    ts = ts_by_dest[ts_by_dest['destination_country'] == dest]
    plt.plot(ts['date'], ts['MAILITM_FID'], marker='o', label=dest)

plt.title('Monthly Package count by Destination Country')
plt.xlabel('Date')
plt.ylabel('Number of Packages')
plt.legend(title='Destination Country')
plt.tight_layout()
plt.show()


### 2. parsed_receptacle_df

In [ ]:
# Group the parsed_receptacle_df by date and destination_country to count receptacles per month per destination country
parsed_receptacle_df['date'] = pd.to_datetime(parsed_receptacle_df['date'])

# Group by destination_country and resample by month 
ts_by_dest_recept = parsed_receptacle_df.groupby(['destination_country', pd.Grouper(key='date', freq='ME')])['RECPTCL_FID'].count().reset_index()

# Choose top 5 destination countries by total count for plotting
top_dest_recept = parsed_receptacle_df['destination_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for dest in top_dest_recept:
    ts = ts_by_dest_recept[ts_by_dest_recept['destination_country'] == dest]
    plt.plot(ts['date'], ts['RECPTCL_FID'], marker='o', label=dest)

plt.title('Monthly Receptacle count by Destination Country')
plt.xlabel('Date')
plt.ylabel('Number of Receptacles')
plt.legend(title='Destination Country')
plt.tight_layout()
plt.show()


## Creating additional features 
 1. 'flow_type' column with values: 'inbound' (to DZ), 'outbound' (from DZ), 'local' (DZ to DZ), otherwise 'other'

In [ ]:
import numpy as np

def get_flow_type(df):
    # Define the conditions
    conditions = [
        (df['destination_country'] == 'DZ') & (df['origin_country'] == 'DZ'), # local
        (df['destination_country'] == 'DZ'),                                # inbound
        (df['origin_country'] == 'DZ')                                     # outbound
    ]
    
    # Define the results for each condition
    choices = ['local', 'inbound', 'outbound']
    
    # Apply logic with 'other' as the default
    return np.select(conditions, choices, default='other')

# Apply to both DataFrames instantly
parsed_packages_df['flow_type'] = get_flow_type(parsed_packages_df)
parsed_receptacle_df['flow_type'] = get_flow_type(parsed_receptacle_df)

# Print counts
print("Flow type counts in parsed_packages_df:\n", parsed_packages_df['flow_type'].value_counts())
print("\nFlow type counts in parsed_receptacle_df:\n", parsed_receptacle_df['flow_type'].value_counts())

we can see that there are some values of flow type with the type "other"

In [ ]:
# Display the first 10 rows of origin_country and destination_country for flow_type 'other' in parsed_receptacle_df
print(parsed_receptacle_df.loc[parsed_receptacle_df['flow_type'] == 'other', ['origin_country', 'destination_country']].head(10))



## Analysis of the relation between the flow_type and the event_type

## 1. parsed_packages_df

In [ ]:

# Analyse relation between flow_type and EVENT_TYPE_CD in parsed_packages_df

plt.figure(figsize=(14,7))
ax = sns.countplot(data=parsed_packages_df, x="EVENT_TYPE_CD", hue="flow_type", palette='Set2')
plt.title("Distribution of EVENT_TYPE_CD by flow_type in parsed_packages_df")
plt.xlabel("EVENT_TYPE_CD")
plt.ylabel("Count")
plt.legend(title="Flow Type")
plt.tight_layout()
for c in ax.containers:
    ax.bar_label(c, label_type='edge', fontsize=8, padding=2)
plt.show()



## 2. parsed_receptacle_df

In [ ]:
# For each flow_type, list the unique EVENT_TYPE_CD values and the most frequent EVENT_TYPE_CD value
for flow in parsed_receptacle_df['flow_type'].unique():
    event_types = parsed_receptacle_df.loc[parsed_receptacle_df['flow_type'] == flow, 'EVENT_TYPE_CD'].unique()
    most_common_event_type = parsed_receptacle_df.loc[parsed_receptacle_df['flow_type'] == flow, 'EVENT_TYPE_CD'].mode()
    print(f"Flow type: {flow}")
    print(f"EVENT_TYPE_CD values: {sorted(event_types)}")
    if not most_common_event_type.empty:
        print(f"Most frequent EVENT_TYPE_CD: {most_common_event_type.iloc[0]}")
    else:
        print("No EVENT_TYPE_CD available")
    print()   


In [ ]:
plt.figure(figsize=(14,7))
ax = sns.countplot(data=parsed_receptacle_df, x="EVENT_TYPE_CD", hue="flow_type", palette='Set1')
plt.title("Distribution of EVENT_TYPE_CD by flow_type in parsed_receptacle_df")
plt.xlabel("EVENT_TYPE_CD")
plt.ylabel("Count")
plt.legend(title="Flow Type")
plt.tight_layout()

for c in ax.containers:
    ax.bar_label(c, label_type='edge', fontsize=8, padding=2)

plt.show()


we can see that there are major event types related to the inbound flow type( coming to DZ)

# Track multiple receptacles just to see the flow of events

In [ ]:
# Track multiple receptacles: visualize all events for 5 different RECPTCL_FID in parsed_receptacle_df
# Pick 5 unique RECPTCL_FID values to demonstrate
num_examples = 5
example_receptacle_ids = parsed_receptacle_df['RECPTCL_FID'].drop_duplicates().iloc[:num_examples]
for rid in example_receptacle_ids:
    print("\n--- Events for RECPTCL_FID:", rid, "---")
    display(parsed_receptacle_df[parsed_receptacle_df['RECPTCL_FID'] == rid][['RECPTCL_FID', 'date', 'EVENT_TYPE_CD', 'etablissement_postal', 'next_etablissement_postal']].sort_values('date').reset_index(drop=True))


plt.figure(figsize=(14, 5))

for i, receptacle_id in enumerate(example_receptacle_ids):
    ex_df = parsed_receptacle_df[parsed_receptacle_df['RECPTCL_FID'] == receptacle_id].sort_values('date')
    plt.plot(
        ex_df['date'],
        ex_df['EVENT_TYPE_CD'],
        marker='o',
        label=f"RECPTCL_FID: {receptacle_id}"
    )

plt.xticks(rotation=45)
plt.title(f"Event Timeline (EVENT_TYPE_CD) for {num_examples} Receptacles")
plt.xlabel("Date")
plt.ylabel("EVENT_TYPE_CD")
plt.tight_layout()
plt.legend()
plt.show()


# calculating the time of existence for each receptacle/ package 
   ## processing_duration 
   Note : this duration is calculated between min and make date and doesn't give the exact duration since it is not suming the duration between each event
   further analysis could be conducted to take that ninto consideration

### 1. parsed_packages_df

In [ ]:
# Calculate processing_duration (time from first to last event per MAILITM_FID)
# For each mail item, sort events by date, get min and max date per mail item
pkg_duration_df = (
    parsed_packages_df
    .sort_values(['MAILITM_FID', 'date'])
    .groupby('MAILITM_FID')['date']
    .agg(['min', 'max'])
    .reset_index()
)
pkg_duration_df.columns = ['MAILITM_FID', 'min_date', 'max_date']
pkg_duration_df['processing_duration'] = pkg_duration_df['max_date'] - pkg_duration_df['min_date']

# Merge 'processing_duration' back to parsed_packages_df
parsed_packages_df = parsed_packages_df.merge(
    pkg_duration_df[['MAILITM_FID', 'processing_duration']],
    on='MAILITM_FID',
    how='left'
)


display results

In [ ]:

# Display processing_duration statistics
print(parsed_packages_df[['MAILITM_FID', 'date', 'processing_duration']].drop_duplicates(subset=['processing_duration']).head(10))
#print the max value of processing_duration
max_duration = parsed_packages_df['processing_duration'].max()
#print the mail item(s) with the max processing_duration
max_duration_items = parsed_packages_df[parsed_packages_df['processing_duration'] == max_duration]['MAILITM_FID'].unique()
print(f"\nMaximum processing duration: {max_duration}")
print(f"Mail item(s) with maximum processing duration: {max_duration_items}")

#print the min value of processing_duration
min_duration = parsed_packages_df['processing_duration'].min()
#print the mail item(s) with the min processing_duration
min_duration_items = parsed_packages_df[parsed_packages_df['processing_duration'] == min_duration]['MAILITM_FID'].unique()
print(f"\nMinimum processing duration: {min_duration}")
print(f"Mail item(s) with minimum processing duration: {min_duration_items}")

visualization of the distribution of the packages according to the processing duration 

In [ ]:
# Visualize the distribution of packages by processing_duration
plt.figure(figsize=(14, 7))

# Convert processing_duration to total days for better visualization
parsed_packages_df['processing_duration_days'] = parsed_packages_df['processing_duration'].dt.total_seconds() / (24 * 3600)

# Create histogram
plt.hist(parsed_packages_df['processing_duration_days'], bins=100, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Processing Duration (days)', fontsize=11)
plt.ylabel('Number of Packages', fontsize=11)
plt.title('Distribution of Packages by Processing Duration', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary statistics
print("Processing Duration Statistics:")
print(f"Mean: {parsed_packages_df['processing_duration_days'].mean():.2f} days")
print(f"Median: {parsed_packages_df['processing_duration_days'].median():.2f} days")
print(f"Std Dev: {parsed_packages_df['processing_duration_days'].std():.2f} days")
print(f"Min: {parsed_packages_df['processing_duration_days'].min():.2f} days")
print(f"Max: {parsed_packages_df['processing_duration_days'].max():.2f} days")

- This doesn't give a lot of insights, we try to analyse the event type and the duration 

In [ ]:
# Summary statistics for processing_duration by EVENT_TYPE_CD
duration_by_event = (
    parsed_packages_df
    .groupby("EVENT_TYPE_CD")['processing_duration_days']
    .describe()
)
print("Processing Duration Statistics by EVENT_TYPE_CD:")
print(duration_by_event)
print("\n")

# Visualize boxplot of processing_duration by EVENT_TYPE_CD
plt.figure(figsize=(14, 7))
sns.boxplot(x='EVENT_TYPE_CD', y='processing_duration_days', data=parsed_packages_df, showfliers=False)
plt.title('Processing Duration by EVENT_TYPE_CD', fontsize=14, fontweight='bold')
plt.xlabel('EVENT_TYPE_CD', fontsize=12)
plt.ylabel('Processing Duration (days)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

- Bar chart showing mean processing_duration by EVENT_TYPE_CD

In [ ]:
mean_duration_by_event = parsed_packages_df.groupby('EVENT_TYPE_CD')['processing_duration_days'].mean().sort_values(ascending=False)
plt.figure(figsize=(14, 7))
mean_duration_by_event.plot(kind='barh', color='steelblue')
plt.xlabel('Mean Processing Duration (days)', fontsize=11)
plt.ylabel('EVENT_TYPE_CD', fontsize=11)
plt.title('Mean Processing Duration by EVENT_TYPE_CD', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()


now we can see that some events type is a kind of groups of similar event types that have approximatly the same average number of processing duration 

## 2. parsed_receptacle_df

In [ ]:
# Calculate processing_duration (time from first to last event per RECPTCL_FID)
# For each receptacle, sort events by date, get min and max date per receptacle
rec_duration_df = (
    parsed_receptacle_df
    .sort_values(['RECPTCL_FID', 'date'])
    .groupby('RECPTCL_FID')['date']
    .agg(['min', 'max'])
    .reset_index()
)
rec_duration_df['processing_duration'] = rec_duration_df['max'] - rec_duration_df['min']

# Merge 'processing_duration' back to parsed_receptacle_df
parsed_receptacle_df = parsed_receptacle_df.merge(
    rec_duration_df[['RECPTCL_FID', 'processing_duration']],
    on='RECPTCL_FID',
    how='left'
)
# Display the first few rows to show the obtained processing_duration values
print(parsed_receptacle_df[['RECPTCL_FID', 'date', 'processing_duration']].head(10))



### Analysis of the relation between the processing duration and the event type 

In [ ]:
# Analysis of processing_duration by EVENT_TYPE_CD
parsed_receptacle_df['processing_duration_hours'] = parsed_receptacle_df['processing_duration'].dt.total_seconds() / 3600
# Summary statistics for processing_duration by EVENT_TYPE_CD
duration_by_event = (
    parsed_receptacle_df
    .groupby("EVENT_TYPE_CD")['processing_duration_hours']
    .describe()
)
print("Processing Duration Statistics by EVENT_TYPE_CD:")
print(duration_by_event)
print("\n")


# Visualize boxplot of processing_duration by EVENT_TYPE_CD
plt.figure(figsize=(14, 7))
sns.boxplot(x='EVENT_TYPE_CD', y='processing_duration_hours', data=parsed_receptacle_df, showfliers=False)
plt.title('Processing Duration by EVENT_TYPE_CD', fontsize=14, fontweight='bold')
plt.xlabel('EVENT_TYPE_CD', fontsize=12)
plt.ylabel('Processing Duration (hours)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

-  Bar chart showing mean processing_duration by EVENT_TYPE_CD

In [ ]:
mean_duration_by_event = parsed_receptacle_df.groupby('EVENT_TYPE_CD')['processing_duration_hours'].mean().sort_values(ascending=False)
plt.figure(figsize=(14, 7))
mean_duration_by_event.plot(kind='barh', color='steelblue')
plt.xlabel('Mean Processing Duration (hours)', fontsize=11)
plt.ylabel('EVENT_TYPE_CD', fontsize=11)
plt.title('Mean Processing Duration by EVENT_TYPE_CD', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()




-  Heatmap: EVENT_TYPE_CD vs Flow Type with mean duration

In [ ]:

pivot_table = parsed_receptacle_df.pivot_table(
    values='processing_duration_hours',
    index='EVENT_TYPE_CD',
    columns='flow_type',
    aggfunc='mean'
)
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt='.1f', cmap='RdYlGn_r', cbar_kws={'label': 'Mean Duration (hours)'})
plt.title('Mean Processing Duration: EVENT_TYPE_CD vs Flow Type', fontsize=14, fontweight='bold')
plt.xlabel('Flow Type', fontsize=12)
plt.ylabel('EVENT_TYPE_CD', fontsize=12)
plt.tight_layout()
plt.show()

- Correlation: Count events per receptacle and duration

In [ ]:
event_count_per_receptacle = parsed_receptacle_df.groupby('RECPTCL_FID').size().reset_index(name='event_count')
receptacle_duration = parsed_receptacle_df[['RECPTCL_FID', 'processing_duration_hours']].drop_duplicates()
event_duration_corr = event_count_per_receptacle.merge(receptacle_duration, on='RECPTCL_FID')

plt.figure(figsize=(12, 7))
plt.scatter(event_duration_corr['event_count'], event_duration_corr['processing_duration_hours'], alpha=0.5, s=30)
plt.xlabel('Number of Events per Receptacle', fontsize=11)
plt.ylabel('Processing Duration (hours)', fontsize=11)
plt.title('Relationship between Event Count and Processing Duration', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation coefficient
correlation = event_duration_corr['event_count'].corr(event_duration_corr['processing_duration_hours'])
print(f"\nCorrelation between event count and processing duration: {correlation:.4f}")

we can see that having more events reduces the duration

### Analysis of the relation between the processing_duration and flow_type

In [ ]:
# We'll analyze the relationship between processing_duration and flow_type .

duration_by_flow = (
    parsed_receptacle_df
    .groupby("flow_type")['processing_duration']
    .describe()
)
print(duration_by_flow)

# Visualize the distribution: boxplot of processing_duration by flow_type

# Convert processing_duration to total hours for visualization
#parsed_receptacle_df['processing_duration_hours'] = parsed_receptacle_df['processing_duration'].dt.total_seconds() / 3600

plt.figure(figsize=(10,6))
sns.boxplot(x='flow_type', y='processing_duration_hours', data=parsed_receptacle_df, showfliers=False)

plt.title('Processing Duration by Flow Type')
plt.xlabel('Flow Type')
plt.ylabel('Processing Duration (hours)')
plt.tight_layout()
plt.show()


From this boxplot of Processing Duration by Flow Type, we can extract several clear results and insights:

1. Inbound flow

Highest processing times overall

Median is relatively high (around ~110–120 hours)

Very wide spread (high variability) → large IQR and long whiskers

Extreme delays exist (outliers reaching ~450+ hours)

2. Outbound flow

Moderate processing time

Median lower than inbound (~80–90 hours)

Still variable, but less extreme than inbound

Some long delays, but fewer and smaller than inbound


3. Local flow

Very low processing duration

Tight distribution (low variance)

Almost no outliers

Local receptacles are processed quickly and consistently.

4. Other flow

Near-zero processing duration

Almost no variability

### Show the row with the max value of processing_duration

In [ ]:
max_row = rec_duration_df.loc[rec_duration_df['processing_duration'].idxmax()]
print(max_row)


In [ ]:
rows = parsed_receptacle_df.loc[parsed_receptacle_df['RECPTCL_FID'] == 'FRCDGADZALGAAEN99999999990102', ['RECPTCL_FID', 'date']]
print(rows)


this is probably an error in the typing since they are the only values existing of the receptacle and the three are from march the diff is in the year of the third

# extracting more features

## receptacle dataset

intuitively we think that the number of etablissement crossed by the receptacle can affect the processing duration

In [ ]:
#calculate the number of unique etablissement crossed by each receptacle
etab_counts = (
    parsed_receptacle_df
    .groupby('RECPTCL_FID')['etablissement_postal']
    .nunique()
    .reset_index()
    .rename(columns={'etablissement_postal': 'num_etablissements'})
)
# Merge back to parsed_receptacle_df
parsed_receptacle_df = parsed_receptacle_df.merge(
    etab_counts,
    on='RECPTCL_FID',
    how='left'
)

parsed_receptacle_df.head(10)

In [ ]:
#calculate the correlation between num_etablissements and processing_duration
correlation = parsed_receptacle_df[['num_etablissements', 'processing_duration_hours']].corr().iloc[0,1]
print(f"Correlation between number of etablissments and processing duration (hours): {correlation}")

considering the size of the dataset this correlation is not bad

In [ ]:
# Visualize the relationship between num_etablissements and processing_duration
plt.figure(figsize=(10,6))
sns.lineplot(
    data=parsed_receptacle_df,
    x='num_etablissements',
    y='processing_duration_hours',
    marker='o'
)
plt.title('Processing Duration vs. Number of Etablissements Crossed')
plt.xlabel('Number of Etablissements Crossed')
plt.ylabel('Processing Duration (hours)')
plt.tight_layout()
plt.show()

the line plot shows a clear increasing trend indicating that as the number of etablissments crossed increases the processing duration also increases, but the correlation value shows that the relation is weak

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='num_etablissements', y='processing_duration_hours', data=parsed_receptacle_df, showfliers=False)
plt.xlabel("Number of Etablissements Crossed")
plt.ylabel("Processing Duration (hours)")
plt.title("Boxplot showing the variation")
plt.show()

by looking at the boxplot we can see that the median processing duration increases with the number of etablissments crossed but the variance also increases significantly,this explains the weak correlation value obtained earlier

## packages dataset

we test the same hypothesis for the packages dataset

In [ ]:
#calculate the number of unique etablissement crossed by each package
etab_counts_pkg = (
    parsed_packages_df
    .groupby('MAILITM_FID')['etablissement_postal']
    .nunique()
    .reset_index()
    .rename(columns={'etablissement_postal': 'num_etablissements'})
)
# Merge back to parsed_packages_df
parsed_packages_df = parsed_packages_df.merge(
    etab_counts_pkg,
    on='MAILITM_FID',
    how='left'
)

In [ ]:
#calculate the correlation between num_etablissements and processing_duration
correlation_pkg = parsed_packages_df[['num_etablissements', 'processing_duration_days']].corr().iloc[0,1]
print(f"Correlation between number of etablissments and processing duration (days): {correlation_pkg}")

In [ ]:
# Visualize the relationship between num_etablissements and processing_duration
plt.figure(figsize=(10,6))
sns.lineplot(
    data=parsed_packages_df,
    x='num_etablissements',
    y='processing_duration_days',
    marker='o'
)
plt.title('Processing Duration vs. Number of Etablissements Crossed')
plt.xlabel('Number of Etablissements Crossed')
plt.ylabel('Processing Duration (days)')
plt.tight_layout()
plt.show()

the correlation is weaker than that of the receptacle dataset this is probably because of variation in packages types

In [ ]:
# Boxplot of processing_duration by num_etablissements
plt.figure(figsize=(10, 6))
sns.boxplot(x='num_etablissements', y='processing_duration_days', data=parsed_packages_df, showfliers=False)
plt.xlabel("Number of Etablissements Crossed")
plt.ylabel("Processing Duration (days)")
plt.title("Boxplot showing the variation")
plt.show()

relation starts to appear more when the number of etablissments crossed is more than 6

In [ ]:
# Merge datasets on RECPTCL_FID to analyze relationship
# merge on left so that all packages are kept, even if their receptacle information is missing
#load the interim files
# parsed_packages_df = pd.read_csv(output_dir / 'interim_parsed_packages_df.csv')
# parsed_receptacle_df = pd.read_csv(output_dir / 'interim_parsed_receptacle_df.csv')

#ensure both are ordered by date using with relation with the RECPTCL_FID
parsed_packages_df = parsed_packages_df.sort_values(by=['RECPTCL_FID', 'date'])
parsed_receptacle_df = parsed_receptacle_df.sort_values(by=['RECPTCL_FID', 'date'])

merged_df = pd.merge(
    parsed_packages_df,
    parsed_receptacle_df,
    on='RECPTCL_FID',
    how='inner',
    suffixes=('_package', '_receptacle')
)

In [ ]:
from pathlib import Path

# Define output directory for saving processed data
output_dir = Path('../data/interim')

# Save merged dataframe
merged_df.to_csv(output_dir / 'interim_merged_packages_receptacle_df.csv', index=False)

### Feature Engineering for Clustering & Anomaly Detection

Before splitting the data, we create new features that will be useful for clustering and anomaly detection. These features are derived from the merged dataset and include:

**Temporal Features:**
- `hour`: Hour of day when package was processed
- `day_of_week`: Day of the week (Sunday to Thursday )
- `is_weekend`: Binary flag for weekend (Friday/Saturday based on our local calendar)

**Package-Level Features:**
- `delay_flag`: Binary flag for packages exceeding 15-day processing threshold
- `delay_per_etab`: Normalized delay accounting for number of establishments
- `pkg_route_step`: Route identifier (current → next establishment)
- `pkg_route_freq`: How common each package route is
- `current_etab_freq`, `next_etab_freq`: Establishment frequency in routing

**Receptacle-Level Aggregated Features:**
- `num_packages`: Number of packages per receptacle
- `avg_processing_days`, `std_processing_days`: Processing time statistics
- `avg_delay_per_etab`: Average normalized delay
- `avg_pkg_route_rarity`: Average rarity of package routes
- `rec_route_freq`: Receptacle route frequency
- `flow_type_freq`: Flow type frequency

In [ ]:

# Ensure date column is datetime
merged_df['date_package'] = pd.to_datetime(merged_df['date_package'])

# Extract temporal features
merged_df['hour'] = merged_df['date_package'].dt.hour
merged_df['day_of_week'] = merged_df['date_package'].dt.dayofweek
merged_df['is_weekend'] = merged_df['day_of_week'].isin([4, 5]).astype(int)  # Friday=4, Saturday=5

print("Temporal features created:")
print(f"  - hour: {merged_df['hour'].min()} to {merged_df['hour'].max()}")
print(f"  - day_of_week: {merged_df['day_of_week'].value_counts().to_dict()}")
print(f"  - is_weekend: {merged_df['is_weekend'].value_counts().to_dict()}")

In [ ]:

# Binary flag for packages exceeding 15-day processing threshold
merged_df['delay_flag'] = (merged_df['processing_duration_days'] > 15).astype(int)

# Normalize processing delay by number of establishments 
# (accounts for packages taking longer routes)
merged_df['delay_per_etab'] = (
    merged_df['processing_duration_days'] / 
    (merged_df['num_etablissements_package'] + 1)
)

print("Delay features created:")
print(f"  - delay_flag: {merged_df['delay_flag'].value_counts().to_dict()}")
print(f"  - delay_per_etab: mean={merged_df['delay_per_etab'].mean():.2f}, std={merged_df['delay_per_etab'].std():.2f}")

In [ ]:

# Create package route identifier (current → next establishment)
merged_df['pkg_route_step'] = (
    merged_df['etablissement_postal_package'] + '→' + 
    merged_df['next_etablissement_postal_package']
)

# Calculate package route frequency (how common each route is)
pkg_route_freq = merged_df['pkg_route_step'].value_counts(normalize=True)
merged_df['pkg_route_freq'] = merged_df['pkg_route_step'].map(pkg_route_freq)

# Calculate establishment frequency (how often each establishment appears)
etab_freq = pd.concat([
    merged_df['etablissement_postal_package'],
    merged_df['next_etablissement_postal_package']
]).value_counts()

merged_df['current_etab_freq'] = merged_df['etablissement_postal_package'].map(etab_freq)
merged_df['next_etab_freq'] = merged_df['next_etablissement_postal_package'].map(etab_freq)

print("Route and frequency features created:")
print(f"  - Unique package routes: {merged_df['pkg_route_step'].nunique()}")
print(f"  - pkg_route_freq: min={merged_df['pkg_route_freq'].min():.6f}, max={merged_df['pkg_route_freq'].max():.4f}")
print(f"  - current_etab_freq: mean={merged_df['current_etab_freq'].mean():.0f}")
print(f"  - next_etab_freq: mean={merged_df['next_etab_freq'].mean():.0f}")

In [ ]:

# Create receptacle route identifier
merged_df['rec_route_step'] = (
    merged_df['etablissement_postal_receptacle'] + '→' + 
    merged_df['next_etablissement_postal_receptacle']
)

# Aggregate package statistics at receptacle level
receptacle_route_stats = (
    merged_df
    .groupby('RECPTCL_FID')
    .agg(
        rec_route=('rec_route_step', 'first'),
        num_packages=('MAILITM_FID', 'count'),
        avg_processing_days=('processing_duration_days', 'mean'),
        std_processing_days=('processing_duration_days', 'std'),
        avg_delay_per_etab=('delay_per_etab', 'mean'),
        avg_pkg_route_rarity=('pkg_route_freq', 'mean')
    )
    .reset_index()
)

# Fill NaN standard deviations (for receptacles with single packages)
receptacle_route_stats['std_processing_days'] = receptacle_route_stats['std_processing_days'].fillna(0)

print("Receptacle aggregated features created:")
print(f"  - Total receptacles: {len(receptacle_route_stats)}")
print(f"  - num_packages: mean={receptacle_route_stats['num_packages'].mean():.2f}")
print(f"  - avg_processing_days: mean={receptacle_route_stats['avg_processing_days'].mean():.2f}")

In [ ]:

# Calculate receptacle route frequency
rec_route_freq = receptacle_route_stats['rec_route'].value_counts(normalize=True)
receptacle_route_stats['rec_route_freq'] = receptacle_route_stats['rec_route'].map(rec_route_freq)

# Map flow type frequencies to each receptacle
rec_flow_type = merged_df.groupby('RECPTCL_FID')['flow_type_receptacle'].first().reset_index()
flow_type_freq = rec_flow_type['flow_type_receptacle'].value_counts(normalize=True)
rec_flow_type['flow_type_freq'] = rec_flow_type['flow_type_receptacle'].map(flow_type_freq)

# Merge flow type frequency back to receptacle stats
receptacle_route_stats = receptacle_route_stats.merge(
    rec_flow_type[['RECPTCL_FID', 'flow_type_freq']],
    on='RECPTCL_FID',
    how='left'
)

print("Receptacle frequency features created:")
print(f"  - Unique receptacle routes: {receptacle_route_stats['rec_route'].nunique()}")
print(f"  - rec_route_freq: min={receptacle_route_stats['rec_route_freq'].min():.6f}, max={receptacle_route_stats['rec_route_freq'].max():.4f}")
print(f"  - flow_type_freq distribution: {receptacle_route_stats['flow_type_freq'].describe()}")

In [ ]:

# Merge receptacle-level aggregated features back to the main dataframe
merged_df = merged_df.merge(
    receptacle_route_stats[[
        'RECPTCL_FID', 'num_packages', 'avg_processing_days', 
        'std_processing_days', 'avg_delay_per_etab', 'avg_pkg_route_rarity',
        'rec_route_freq', 'flow_type_freq'
    ]],
    on='RECPTCL_FID',
    how='left',
    suffixes=('', '_rec')
)

print("Merged receptacle features back to main dataframe")
print(f"Total columns after feature engineering: {len(merged_df.columns)}")
print(f"\nNew features added:")
new_features = ['hour', 'day_of_week', 'is_weekend', 'delay_flag', 'delay_per_etab',
                'pkg_route_step', 'pkg_route_freq', 'current_etab_freq', 'next_etab_freq',
                'rec_route_step', 'num_packages', 'avg_processing_days', 'std_processing_days',
                'avg_delay_per_etab', 'avg_pkg_route_rarity', 'rec_route_freq', 'flow_type_freq']
for feat in new_features:
    if feat in merged_df.columns:
        print(f"   {feat}")

In [ ]:

# Package-level features for clustering
pkg_features = [
    'processing_duration_days',
    'delay_flag',
    'delay_per_etab',
    'num_etablissements_package',
    'pkg_route_freq',
    'current_etab_freq',
    'next_etab_freq',
    'hour',
    'is_weekend'
]

# Receptacle-level features for clustering
rec_features = [
    'num_packages',
    'avg_processing_days',
    'std_processing_days',
    'avg_delay_per_etab',
    'avg_pkg_route_rarity',
    'rec_route_freq',
    'flow_type_freq'
]

# All numeric features for clustering (combined)
numeric_clustering_features = pkg_features + rec_features

# Check for NaN values in clustering features
print("Checking for NaN values in clustering features:")
nan_counts = merged_df[numeric_clustering_features].isna().sum()
nan_counts = nan_counts[nan_counts > 0]
if len(nan_counts) > 0:
    print("Features with NaN values:")
    print(nan_counts)
else:
    print("   No NaN values found in clustering features")

print(f"\nPackage-level features: {len(pkg_features)}")
print(f"Receptacle-level features: {len(rec_features)}")
print(f"Total numeric features for clustering: {len(numeric_clustering_features)}")

## Data Splitting to training and testing sets
- We propose to split datasets (which are already sorted by date) using `TimeSeriesSplit` from `sklearn.model_selection`
- because we need to keep all rows for a specific `MAILITM_FID` as a single block and not separated.
- older rows are kept for the `training_set` and newer ones are kept for the `testing_set`.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Sort by date to ensure proper time series splitting
merged_df_sorted = merged_df.sort_values(by='date_package').reset_index(drop=True)

# Identify feature types for the preprocessor
# Categorical features to one-hot encode
cat_features_for_split = [
    'origin_country_package', 'destination_country_package',
    'origin_country_receptacle', 'destination_country_receptacle',
    'flow_type_package', 'flow_type_receptacle'
]
# Filter to only include features that exist
cat_features = [col for col in cat_features_for_split if col in merged_df_sorted.columns]

# Numeric features to scale (including our engineered features)
numeric_features_for_clustering = numeric_clustering_features.copy()

print(f"Categorical features for encoding: {len(cat_features)}")
print(f"Numeric features for scaling: {len(numeric_features_for_clustering)}")
print(f"Total features: {len(merged_df_sorted.columns)}")

# Create preprocessor for the merged dataset with engineered features
merged_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('num', StandardScaler(), numeric_features_for_clustering)
    ],
    remainder='drop'  # Drop columns not specified (like dates, IDs, route strings)
)

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit()

# Display split information
print(f"\nTimeSeriesSplit configuration:")
for fold, (train_idx, test_idx) in enumerate(tscv.split(merged_df_sorted)):
    print(f"Fold {fold + 1}: Train size={len(train_idx)}, Test size={len(test_idx)}")

In [ ]:
def get_etab_mapping_from_merged(df, source='package'):
 
    # Select appropriate column suffixes based on source
    suffix = f'_{source}'
    
    # Get the columns we need
    etab_col = f'etablissement_postal{suffix}'
    next_etab_col = f'next_etablissement_postal{suffix}'
    
    # Verify columns exist
    if etab_col not in df.columns or next_etab_col not in df.columns:
        print(f"Warning: Columns {etab_col} or {next_etab_col} not found in dataframe")
        print(f"Available columns: {[c for c in df.columns if 'etablissement' in c.lower()]}")
        return {}
    
    # 1. Calculate the global mode once (to use as a safe fallback)
    # Filter out NaN values for mode calculation
    mode_values = df[next_etab_col].dropna()
    if len(mode_values) > 0:
        global_mode = mode_values.mode().iat[0] if not mode_values.mode().empty else None
    else:
        global_mode = None
    
    # 2. Get the mode for every group at once
    # This creates a Series where index = etablissement, value = most frequent next
    modes_per_group = df.groupby(etab_col)[next_etab_col].agg(
        lambda x: x.mode().iat[0] if not x.mode().empty else global_mode
    )
    
    # 3. Convert to dictionary
    etablissement_dict = modes_per_group.to_dict()
    
    return etablissement_dict





* After splitting, for each pair `(X_train, X_test)` of `packages_splits`, we'll fill the null values of `next_etablissement_postal` based on the most frequent value of `next_etablissement_postal` appearing with the value of `etablissement_postal` of each specific row having a null value in `next_etablissement_postal`
* Example:
say that a row has a null `next_etablissement_postal`, we look at it's `etablissement_postal` value (say `VAL`), we iterate through the training set, we count how many times each `ETAB_XXXX` in ` next_etablissement_postal` appears with `VAL` being in `etablissement_postal`, we take the mode, and we use it fill all rows having null value at `next_etablissement_postal` where their `etablissement_postal` is `VAL`

In [ ]:
def build_splits(preprocessor, df, id_col, date_col='date_package'):

    # Sort by date
    df = df.sort_values(date_col).reset_index(drop=True)
    
    # Get unique IDs for splitting
    unique_ids = df[id_col].unique()
    
    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    splits = []
    
    # Define both sources to process
    sources = ['package', 'receptacle']
 
    for fold_num, (train_id_idx, test_id_idx) in enumerate(tscv.split(unique_ids)):
        # 1. Get the actual IDs for this fold
        train_ids = unique_ids[train_id_idx]
        test_ids = unique_ids[test_id_idx]
        
        # 2. Filter the main dataframe to get rows for this fold
        train_df = df[df[id_col].isin(train_ids)].copy()
        test_df = df[df[id_col].isin(test_ids)].copy()
        
        # 3. Handle NULL VALUES for BOTH package and receptacle columns
        for source in sources:
            etab_col = f'etablissement_postal_{source}'
            next_etab_col = f'next_etablissement_postal_{source}'
            
            # Only process if these columns exist
            if etab_col in train_df.columns and next_etab_col in train_df.columns:
                # Create mapping from training data
                etablissement_dict = get_etab_mapping_from_merged(train_df, source=source)
                
                # Fill NULLs in both train and test sets
                for fold_data in [train_df, test_df]:
                    # Find rows with missing next_etablissement_postal
                    mask = fold_data[next_etab_col].isna()
                    
                    if mask.any():
                        # Map using the dictionary
                        fold_data.loc[mask, next_etab_col] = fold_data.loc[mask, etab_col].map(etablissement_dict)
                        
                        # Fill any remaining NaNs with global mode from training data
                        if fold_data[next_etab_col].isna().any():
                            global_mode = train_df[next_etab_col].dropna().mode()
                            if len(global_mode) > 0:
                                fold_data[next_etab_col].fillna(global_mode.iat[0], inplace=True)
                        
        
        # 4. RECALCULATE FEATURES that depend on next_etablissement_postal for BOTH sides
        for fold_data in [train_df, test_df]:
            # ==================== PACKAGE-side features ====================
            if 'etablissement_postal_package' in fold_data.columns:
                etab_pkg = 'etablissement_postal_package'
                next_etab_pkg = 'next_etablissement_postal_package'
                
                # Recalculate pkg_route_step and pkg_route_freq
                fold_data['pkg_route_step'] = fold_data[etab_pkg] + '→' + fold_data[next_etab_pkg]
                
                # Recalculate next_etab_freq (PACKAGE side) using training data frequencies
                next_etab_freq_pkg = train_df[next_etab_pkg].value_counts(normalize=True)
                fold_data['next_etab_freq'] = fold_data[next_etab_pkg].map(next_etab_freq_pkg)
                fold_data['next_etab_freq'].fillna(next_etab_freq_pkg.min() if len(next_etab_freq_pkg) > 0 else 0.0001, inplace=True)
                
                # Recalculate pkg_route_freq using training data frequencies
                pkg_route_freq = train_df['pkg_route_step'].value_counts(normalize=True)
                fold_data['pkg_route_freq'] = fold_data['pkg_route_step'].map(pkg_route_freq)
                fold_data['pkg_route_freq'].fillna(pkg_route_freq.min() if len(pkg_route_freq) > 0 else 0.0001, inplace=True)
            
            # ==================== RECEPTACLE-side features ====================
            if 'etablissement_postal_receptacle' in fold_data.columns:
                etab_rec = 'etablissement_postal_receptacle'
                next_etab_rec = 'next_etablissement_postal_receptacle'
                
                # Recalculate rec_route_step and rec_route_freq
                fold_data['rec_route_step'] = fold_data[etab_rec] + '→' + fold_data[next_etab_rec]
                
                # # Recalculate next_etab_freq_rec (RECEPTACLE side) using training data frequencies
                # next_etab_freq_rec = train_df[next_etab_rec].value_counts(normalize=True)
                # fold_data['next_etab_freq_rec'] = fold_data[next_etab_rec].map(next_etab_freq_rec)
                # fold_data['next_etab_freq_rec'].fillna(next_etab_freq_rec.min() if len(next_etab_freq_rec) > 0 else 0.0001, inplace=True)
                
                # Recalculate rec_route_freq using training data frequencies
                rec_route_freq = train_df['rec_route_step'].value_counts(normalize=True)
                fold_data['rec_route_freq'] = fold_data['rec_route_step'].map(rec_route_freq)
                fold_data['rec_route_freq'].fillna(rec_route_freq.min() if len(rec_route_freq) > 0 else 0.0001, inplace=True)
    
        
        # 5. Finally, Transform to Numeric using the preprocessor
        X_train = preprocessor.fit_transform(train_df)
        X_test = preprocessor.transform(test_df)
        
        splits.append((X_train, X_test))
    return splits

In [ ]:
# Build splits using the merged dataframe - handles BOTH package and receptacle
print("Building splits for MERGED DATA")
merged_splits = build_splits(
    merged_preprocessor, 
    merged_df_sorted, 
    id_col='RECPTCL_FID',
    date_col='date_package'
)
# Display summary of merged splits
print("Summary of merged splits:")
print(f"Total folds: {len(merged_splits)}")
for i, (X_train, X_test) in enumerate(merged_splits):
    print(f"Fold {i + 1}: X_train shape = {X_train.shape}, X_test shape = {X_test.shape}")

**We still need to deal with the high dimensionality we get from `OneHotEncoder`**

* now each of `packages_splits` and `receptacles_splits` have 5 pairs of `(X_train, X_test)` following the cross-validation method
* for each pair, we'll apply a classification algorithm to predict the clustering and anomaly detection targets for the testing data

# 1. Clustering

## 1.1 KMeans

In [ ]:

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np

# Use the first fold from merged_splits for initial clustering
X_train, X_test = merged_splits[0]

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

print("Finding optimal K")


k_range = range(2, 11)
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_train)
   
    if k > 1:
        sil_score = silhouette_score(X_train, kmeans.labels_, sample_size=min(10000, len(X_train)))
        silhouette_scores.append(sil_score)
        print(f"K={k}: Silhouette={sil_score:.4f}")


fig, axes = plt.subplots(1, 1, figsize=(7, 5))


# Silhouette score plot
axes.plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes.set_xlabel('Number of Clusters (K)', fontsize=12)
axes.set_ylabel('Silhouette Score', fontsize=12)
axes.set_title('Silhouette Score vs Number of Clusters', fontsize=14)
axes.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find best K based on silhouette score
best_k = k_range[np.argmax(silhouette_scores)]
print(f"\n Best K based on silhouette score: {best_k}")

In [ ]:
from sklearn.decomposition import PCA

chosen_k = 10
print(f" CLUSTERING WITH K={chosen_k}")
print("="*60)

# Fit final KMeans model
final_kmeans = KMeans(n_clusters=chosen_k, random_state=42, n_init=10)
train_labels = final_kmeans.fit_predict(X_train)
test_labels = final_kmeans.predict(X_test)

print(f"\nTraining set cluster distribution:")
unique, counts = np.unique(train_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} samples ({count/len(train_labels)*100:.1f}%)")

print(f"\nTest set cluster distribution:")
unique, counts = np.unique(test_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} samples ({count/len(test_labels)*100:.1f}%)")

# Reduce to 2D for visualization
pca = PCA(n_components=2)
X_train_2d = pca.fit_transform(X_train)
X_test_2d = pca.transform(X_test)

print(f"\nPCA explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {sum(pca.explained_variance_ratio_)*100:.2f}%")

# Plot clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Training set
scatter1 = axes[0].scatter(X_train_2d[:, 0], X_train_2d[:, 1], 
                           c=train_labels, cmap='viridis', alpha=0.7, s=50)
axes[0].set_xlabel('PC1', fontsize=12)
axes[0].set_ylabel('PC2', fontsize=12)
axes[0].set_title(f'Training Set Clusters (K={chosen_k})', fontsize=14)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# Test set
scatter2 = axes[1].scatter(X_test_2d[:, 0], X_test_2d[:, 1], 
                           c=test_labels, cmap='viridis', alpha=0.7, s=50)
axes[1].set_xlabel('PC1', fontsize=12)
axes[1].set_ylabel('PC2', fontsize=12)
axes[1].set_title(f'Test Set Clusters (K={chosen_k})', fontsize=14)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

print("CLUSTER QUALITY METRICS")
print("="*60)

# Silhouette Score
sil_train = silhouette_score(X_train, train_labels)
sil_test = silhouette_score(X_test, test_labels)
print(f"Silhouette Score (Train): {sil_train:.4f}")
print(f"Silhouette Score (Test):  {sil_test:.4f}")



## 1.2 DBSCAN 


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import numpy as np


# 2. TRY DIFFERENT EPS VALUES
print("DBSCAN WITH DIFFERENT EPS VALUES")
print("="*60)

# Try multiple eps values around the suggested one
eps_values = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
min_samples = 5

results = []
for eps in eps_values:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_train)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    noise_ratio = n_noise / len(labels) * 100
    
    # Calculate silhouette score only if there are at least 2 clusters and not all noise
    if n_clusters >= 2 and n_noise < len(labels):
        # Exclude noise points for silhouette calculation
        mask = labels != -1
        if mask.sum() > 1:
            sil = silhouette_score(X_train[mask], labels[mask])
        else:
            sil = -1
    else:
        sil = -1
    
    results.append({
        'eps': eps,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'noise_ratio': noise_ratio,
        'silhouette': sil
    })
    
    print(f"eps={eps:.1f}: {n_clusters} clusters, {n_noise} noise points ({noise_ratio:.1f}%), Silhouette={sil:.4f}" if sil != -1 else f"eps={eps:.1f}: {n_clusters} clusters, {n_noise} noise points ({noise_ratio:.1f}%), Silhouette=N/A")

# Find best eps based on silhouette score (among valid ones)
valid_results = [r for r in results if r['silhouette'] > 0]
if valid_results:
    best_result = max(valid_results, key=lambda x: x['silhouette'])
    best_eps = best_result['eps']
    print(f"\n Best eps based on silhouette: {best_eps}")
else:
    best_eps = 1.5  # Default
    print(f"\n No valid clustering found, using default eps: {best_eps}")

In [ ]:
# DBSCAN MODEL AND VISUALIZATION

print(f"FINAL DBSCAN CLUSTERING (eps={best_eps}, min_samples={min_samples})")
print("="*60)

# Fit final DBSCAN model
final_dbscan = DBSCAN(eps=best_eps, min_samples=min_samples)
dbscan_train_labels = final_dbscan.fit_predict(X_train)

# For test set, we need to assign labels based on nearest core points
# DBSCAN doesn't have a predict method, so we use a workaround
from sklearn.neighbors import NearestNeighbors

# Find core samples from training
core_samples_mask = np.zeros_like(dbscan_train_labels, dtype=bool)
core_samples_mask[final_dbscan.core_sample_indices_] = True

# Use nearest neighbors to assign test points to clusters
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_train[core_samples_mask])
distances, indices = nn.kneighbors(X_test)

# Assign test labels based on nearest core point
dbscan_test_labels = np.array([
    dbscan_train_labels[final_dbscan.core_sample_indices_[idx[0]]] 
    if dist[0] <= best_eps else -1 
    for dist, idx in zip(distances, indices)
])

# Results summary
n_clusters_train = len(set(dbscan_train_labels)) - (1 if -1 in dbscan_train_labels else 0)
n_noise_train = list(dbscan_train_labels).count(-1)

n_clusters_test = len(set(dbscan_test_labels)) - (1 if -1 in dbscan_test_labels else 0)
n_noise_test = list(dbscan_test_labels).count(-1)

print(f"\nTraining Set Results:")
print(f"  Number of clusters: {n_clusters_train}")
print(f"  Noise points: {n_noise_train} ({n_noise_train/len(dbscan_train_labels)*100:.1f}%)")
print(f"  Core samples: {len(final_dbscan.core_sample_indices_)}")

print(f"\nTest Set Results:")
print(f"  Number of clusters: {n_clusters_test}")
print(f"  Noise points: {n_noise_test} ({n_noise_test/len(dbscan_test_labels)*100:.1f}%)")

# Cluster distribution
print(f"\nTraining cluster distribution:")
unique, counts = np.unique(dbscan_train_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    label = "Noise" if cluster == -1 else f"Cluster {cluster}"
    print(f"  {label}: {count} samples ({count/len(dbscan_train_labels)*100:.1f}%)")

 
# Use PCA from previous KMeans analysis
X_train_2d = pca.transform(X_train)
X_test_2d = pca.transform(X_test)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Training set - color noise points differently
colors_train = ['gray' if l == -1 else plt.cm.viridis(l / max(dbscan_train_labels.max(), 1)) 
                for l in dbscan_train_labels]
axes[0].scatter(X_train_2d[:, 0], X_train_2d[:, 1], c=colors_train, alpha=0.7, s=50)
axes[0].scatter(X_train_2d[core_samples_mask, 0], X_train_2d[core_samples_mask, 1], 
                c='red', marker='x', s=30, alpha=0.5, label='Core samples')
axes[0].set_xlabel('PC1', fontsize=12)
axes[0].set_ylabel('PC2', fontsize=12)
axes[0].set_title(f'DBSCAN Training Set (eps={best_eps})\n{n_clusters_train} clusters, {n_noise_train} noise', fontsize=14)
axes[0].legend()

# Test set
colors_test = ['gray' if l == -1 else plt.cm.viridis(l / max(dbscan_test_labels.max(), 1)) 
               for l in dbscan_test_labels]
axes[1].scatter(X_test_2d[:, 0], X_test_2d[:, 1], c=colors_test, alpha=0.7, s=50)
axes[1].set_xlabel('PC1', fontsize=12)
axes[1].set_ylabel('PC2', fontsize=12)
axes[1].set_title(f'DBSCAN Test Set\n{n_clusters_test} clusters, {n_noise_test} noise', fontsize=14)

plt.tight_layout()
plt.show()

# DBSCAN VS KMEANS

print("COMPARISON: DBSCAN vs KMeans")


# Calculate metrics for non-noise points
mask_train = dbscan_train_labels != -1
mask_test = dbscan_test_labels != -1

if mask_train.sum() > 1 and len(set(dbscan_train_labels[mask_train])) > 1:
    dbscan_sil_train = silhouette_score(X_train[mask_train], dbscan_train_labels[mask_train])
else:
    dbscan_sil_train = None

if mask_test.sum() > 1 and len(set(dbscan_test_labels[mask_test])) > 1:
    dbscan_sil_test = silhouette_score(X_test[mask_test], dbscan_test_labels[mask_test])
else:
    dbscan_sil_test = None

print(f"\n{'Metric':<30} {'KMeans':>15} {'DBSCAN':>15}")
print("-" * 60)
print(f"{'Number of Clusters (Train)':<30} {chosen_k:>15} {n_clusters_train:>15}")
print(f"{'Noise Points (Train)':<30} {'0':>15} {n_noise_train:>15}")
print(f"{'Silhouette Score (Train)':<30} {sil_train:>15.4f} {dbscan_sil_train:>15.4f}" if dbscan_sil_train else f"{'Silhouette Score (Train)':<30} {sil_train:>15.4f} {'N/A':>15}")
print(f"{'Silhouette Score (Test)':<30} {sil_test:>15.4f} {dbscan_sil_test:>15.4f}" if dbscan_sil_test else f"{'Silhouette Score (Test)':<30} {sil_test:>15.4f} {'N/A':>15}")